In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

In [2]:
results_mpc = pd.read_csv('mpc_results_def.csv')
results_mpc['timestamp'] = pd.to_datetime(results_mpc['timestamp'])

results_nn = pd.read_csv('hybrid_rl_results_def.csv')
results_nn['timestamp'] = pd.to_datetime(results_nn['timestamp'])

start = pd.Timestamp(year=2018,
                     month=10,
                     day=20,
                     hour=0,
                     minute=0)
end = pd.Timestamp(year=2018,
                     month=10,
                     day=30,
                     hour=0,
                     minute=0)

results_mpc_short = results_mpc[(results_mpc['timestamp'] > start) & (results_mpc['timestamp'] < end)]
results_nn_short = results_nn[(results_nn['timestamp'] > start) & (results_nn['timestamp'] < end)]

In [3]:
def compute_reward(results):
    reward = []
    for i in range(len(results)):
        r = (results['battery_bid_energy_power_gen'].iloc[i] - results['battery_bid_energy_power_load'].iloc[i]) * results['battery_bid_energy_price'].iloc[i]
        if results['battery_bid_fcas_price'].iloc[i] <= results['fcas_clearing_price'].iloc[i]:
            r += abs(results['battery_bid_fcas_price'].iloc[i] * results['battery_bid_fcas_power'].iloc[i])
        reward.append(r)
    results['r'] = reward
    return results

In [30]:
results_mpc = compute_reward(results_mpc)

print(results_mpc['r'].sum())
print(results_nn['r'].sum())

-118151964.02958937
73639729.87117867


In [28]:
cumulative_reward_mpc = results_mpc['r'].rolling(window=48).mean()
cumulative_reward_nn = results_nn['r'].rolling(window=48).mean()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results_mpc['timestamp'], y=cumulative_reward_mpc, name='reward MPC'), secondary_y=False)
fig.add_trace(go.Scatter(x=results_nn['timestamp'], y=cumulative_reward_nn, name='reward SAC'), secondary_y=False)


fig.update_layout(
    font=dict(family='Courier New, monospace', size=18, color='black'),
    legend_orientation="h",
    legend=go.layout.Legend(
        x=0.76,
        y=0,
        traceorder="normal",
        font=dict(
            family="Courier New, monospace",
            size=17,
        )),
    )

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Cumulative Reward")
fig.show()

In [29]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results_nn['timestamp'], y=results_nn['k'], name='k'), secondary_y=False)

# Add figure title
fig.update_layout(
    font=dict(family='Courier New, monospace', size=18, color='black'),
    legend_orientation="h",
    legend=go.layout.Legend(
        x=0.76,
        y=0,
        traceorder="normal",
        font=dict(
            family="Courier New, monospace",
            size=17,
        )),
    )

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Risk Factor")

fig.show()